In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from sklearn.metrics import mean_squared_error
import math
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load DataFrame and Libs

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

df = pd.read_csv('Dataframes/3rdBodyPerturbationPoliastro.csv')
df_v = pd.read_csv('Dataframes/3rdBodyPerturbationPoliastro_vv.csv')

df['x_vv']=df_v.vv_x
df['y_vv']=df_v.vv_y
df['z_vv']=df_v.vv_z

df

,x,y,z,x_vv,y_vv,z_vv
0,42159.783600,0.000000,0.000000,0.000000,3.074505,0.053666
1,39159.545914,15563.916303,266.913399,-1.142984,2.852848,0.048528
2,30572.229294,28909.053086,496.928223,-2.127756,2.228986,0.039518
3,17596.068412,38144.740319,671.356184,-2.813645,1.287415,0.027234
4,2068.446366,41949.706443,775.646554,-3.098444,0.161675,0.012698
...,...,...,...,...,...,...
995,38132.961517,21370.880108,-7532.333000,-0.876644,2.199413,1.750026
996,31399.515894,31233.289707,1828.575655,-1.692745,1.562118,1.820311
997,20906.077447,37238.565267,10967.804062,-2.312211,0.726117,1.664753
998,7871.245318,38575.122415,18731.818721,-2.661184,-0.221639,1.293445


In [3]:
# Train-test split

train_size = int(len(df) * 0.8) 

train_df,test_df = df[1:train_size], df[train_size:] 

In [4]:
test_df

,x,y,z,x_vv,y_vv,z_vv
800,-38640.279625,-2764.935183,17604.980941,-0.242379,-2.869481,-1.032661
801,-37222.004154,-17134.306183,11135.984105,0.783117,-2.603604,-1.431914
802,-30699.416577,-29154.049729,3087.546466,1.701144,-1.974296,-1.633559
803,-19981.800403,-37148.488205,-5420.909441,2.380643,-1.071344,-1.607049
804,-6568.003292,-40025.406603,-13203.492438,2.729043,-0.025797,-1.357603
...,...,...,...,...,...,...
995,38132.961517,21370.880108,-7532.333000,-0.876644,2.199413,1.750026
996,31399.515894,31233.289707,1828.575655,-1.692745,1.562118,1.820311
997,20906.077447,37238.565267,10967.804062,-2.312211,0.726117,1.664753
998,7871.245318,38575.122415,18731.818721,-2.661184,-0.221639,1.293445


In [5]:
# Scaling

train = train_df
scalers={}
for i in train_df.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_df
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

In [6]:
# Now we will make a function that will use a sliding window approach to transform our series into samples 
# of input past observations and output future observations to use supervised learning algorithms.

# FUNCTION
def split_series(series, n_past, n_future):
    # n_past ==> no of past observations
    # n_future ==> no of future observations 
    X, y = list(), list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
        # slicing the past and future parts of the window
        past, future = series[window_start:past_end, :], series[past_end:future_end, :]
        X.append(past)
        y.append(future)
    return np.array(X), np.array(y)

In [7]:
# Convert train and test data:

# Variables
n_past = 60
n_future = 5
n_features = 6

# CALLS
X_train, y_train = split_series(train.values, n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))

X_test, y_test = split_series(test.values, n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [8]:
X_train.shape

(735, 60, 6)

In [9]:
X_test.shape

(136, 60, 6)

In [10]:
y_train.shape

(735, 5, 6)

In [11]:
y_test.shape

(136, 5, 6)

# Multivariate model

# Model 1: E1D1 ==> Sequence to Sequence Model with one encoder layer and one decoder layer.

In [12]:
# E1D1
# n_features ==> no of features at each timestep in the data.

encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(128, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]

decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])
decoder_l1 = tf.keras.layers.LSTM(128, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

model = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss = tf.keras.losses.Huber())

history = model.fit(
                    X_train, y_train, 
                    batch_size=256, 
                    epochs=100, 
                    validation_data=(X_test,y_test)
                   )

Epoch 1/100
3/3 [==============================] - 1s 400ms/step - loss: 0.1553 - val_loss: 0.1599
Epoch 2/100
3/3 [==============================] - 0s 105ms/step - loss: 0.1066 - val_loss: 0.1050
Epoch 3/100
3/3 [==============================] - 0s 103ms/step - loss: 0.0625 - val_loss: 0.0660
Epoch 4/100
3/3 [==============================] - 0s 105ms/step - loss: 0.0402 - val_loss: 0.0675
Epoch 5/100
3/3 [==============================] - 0s 96ms/step - loss: 0.0427 - val_loss: 0.0556
Epoch 6/100
3/3 [==============================] - 0s 91ms/step - loss: 0.0297 - val_loss: 0.0400
Epoch 7/100
3/3 [==============================] - 0s 96ms/step - loss: 0.0209 - val_loss: 0.0357
Epoch 8/100
3/3 [==============================] - 0s 87ms/step - loss: 0.0184 - val_loss: 0.0317
Epoch 9/100
3/3 [==============================] - 0s 90ms/step - loss: 0.0141 - val_loss: 0.0263
Epoch 10/100
3/3 [==============================] - 0s 107ms/step - loss: 0.0099 - val_loss: 0.0250
Epoch 11/100
3

Epoch 81/100
3/3 [==============================] - 0s 113ms/step - loss: 2.2334e-04 - val_loss: 6.3242e-04
Epoch 82/100
3/3 [==============================] - 0s 105ms/step - loss: 2.1977e-04 - val_loss: 6.0777e-04
Epoch 83/100
3/3 [==============================] - 0s 106ms/step - loss: 2.2912e-04 - val_loss: 6.1254e-04
Epoch 84/100
3/3 [==============================] - 0s 109ms/step - loss: 2.2923e-04 - val_loss: 6.2922e-04
Epoch 85/100
3/3 [==============================] - 0s 96ms/step - loss: 2.2601e-04 - val_loss: 6.0383e-04
Epoch 86/100
3/3 [==============================] - 0s 100ms/step - loss: 2.0998e-04 - val_loss: 6.2290e-04
Epoch 87/100
3/3 [==============================] - 0s 102ms/step - loss: 2.0794e-04 - val_loss: 5.6625e-04
Epoch 88/100
3/3 [==============================] - 0s 102ms/step - loss: 2.0289e-04 - val_loss: 5.8493e-04
Epoch 89/100
3/3 [==============================] - 0s 110ms/step - loss: 1.9941e-04 - val_loss: 5.7159e-04
Epoch 90/100
3/3 [===========

In [13]:
pred = model.predict(X_test)
pred.shape

(136, 5, 6)

In [14]:
for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]
    pred[:,:,index]=scaler.inverse_transform(pred[:,:,index])
    #y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])
y_test.shape

(136, 5, 6)

In [15]:
y_test[:,-1,0]

array([ -8580.68636836, -21580.77033154, -31508.40559114, -37025.09594659,
       -37423.65401184, -32679.21080377, -23442.55060601, -10978.72035769,
         2985.7054782 ,  16520.84509467,  27798.55840532,  35344.6529504 ,
        38188.59658665,  35937.84933107,  28822.74069404,  17720.63777403,
         4114.25199323, -10076.80861617, -22801.97239595, -32242.02806097,
       -37102.74134816, -36769.24757745, -31336.97455908, -21571.53138319,
        -8818.30767242,   5160.81489587,  18443.01905644,  29245.27551094,
        36173.05174199,  38358.08582002,  35508.90974646,  27930.06425218,
        16532.92597256,   2803.99330865, -11337.25085958, -23827.15132184,
       -32834.79545718, -37090.67245283, -36060.64730559, -29961.60688874,
       -19682.2591247 ,  -6654.86141998,   7325.09001583,  20349.69801356,
        30687.39019716,  37022.46297369,  38582.7040899 ,  35170.55413207,
        27159.65660567,  15491.25244685,   1649.62778429, -12452.37006549,
       -24737.71141331, -

In [16]:
pred[:,-1,0]

array([ -7310.2075, -20416.328 , -30638.822 , -36454.66  , -37127.992 ,
       -32454.846 , -22901.484 , -10072.177 ,   3721.1104,  16675.24  ,
        27673.438 ,  35500.168 ,  38706.43  ,  36395.676 ,  29030.355 ,
        18041.791 ,   4885.9995,  -8964.792 , -21714.18  , -31359.957 ,
       -36495.75  , -36452.727 , -31070.967 , -20919.717 ,  -7772.1304,
         6024.104 ,  18757.268 ,  29344.918 ,  36537.984 ,  38941.195 ,
        35846.695 ,  27904.434 ,  16573.69  ,   3286.797 , -10467.148 ,
       -22863.1   , -31948.688 , -36425.164 , -35683.117 , -29617.576 ,
       -18914.146 ,  -5490.675 ,   8298.477 ,  20823.068 ,  31008.55  ,
        37572.01  ,  39184.23  ,  35333.434 ,  26842.521 ,  15181.983 ,
         1765.6252, -11892.879 , -23933.705 , -32460.344 , -36273.35  ,
       -34820.113 , -28070.35  , -16847.836 ,  -3193.335 ,  10565.612 ,
        22879.076 ,  32659.928 ,  38587.47  ,  39402.707 ,  34798.758 ,
        25765.58  ,  13776.235 ,    230.6917, -13326.247 , -2499

In [17]:
for index,i in enumerate(train_df.columns):
    #print(index)
    print('Feature: '+str(i))
    for j in range(0,n_future):
        print("Step <",j,"> RMSE: ",math.sqrt(mean_squared_error(y_test[:,j-1,index],pred[:,j-1,index])))
        print()

Feature: x
Step < 0 > RMSE:  741.8636113655941

Step < 1 > RMSE:  1461.105774440013

Step < 2 > RMSE:  1171.3283051727487

Step < 3 > RMSE:  930.4651206839704

Step < 4 > RMSE:  891.343571907061

Feature: y
Step < 0 > RMSE:  1687.772864697907

Step < 1 > RMSE:  843.3998473949282

Step < 2 > RMSE:  1406.354374214958

Step < 3 > RMSE:  1217.5692903208433

Step < 4 > RMSE:  1140.4338398821203

Feature: z
Step < 0 > RMSE:  1271.1799591458705

Step < 1 > RMSE:  1189.3627613345889

Step < 2 > RMSE:  916.7499154299568

Step < 3 > RMSE:  940.7466974957858

Step < 4 > RMSE:  989.4400842412625

Feature: x_vv
Step < 0 > RMSE:  0.06431069453822269

Step < 1 > RMSE:  0.09334642694541788

Step < 2 > RMSE:  0.07568697992387693

Step < 3 > RMSE:  0.06292630134831834

Step < 4 > RMSE:  0.07033954918377133

Feature: y_vv
Step < 0 > RMSE:  0.08058858679974493

Step < 1 > RMSE:  0.07054734172955823

Step < 2 > RMSE:  0.09585313620611784

Step < 3 > RMSE:  0.08707497089310015

Step < 4 > RMSE:  0.074436348